In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import h5py
import api as pyq 

/Users/ksu/Software/anaconda3/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
# Wavefunction generation
from scf_runs import run_lda_he, run_casci
scf_checkfile, opt_checkfile, mf_lda = run_lda_he()
ci_checkfile, mc = run_casci(scf_checkfile, nroots=4)

/Users/ksu/Documents/GitHub/pyqmc/pyqmc/api.py
/Users/ksu/Software/anaconda3/lib/python3.11/site-packages/pyscf/__init__.py
He atom neutral LDA spin=0
HF
converged SCF energy = -2.85516047724274  <S^2> = 0  2S+1 = 1
LDA
converged SCF energy = -2.82670655418611  <S^2> = 4.4408921e-16  2S+1 = 1


In [14]:
import importlib, slater, pyscftools,recipes, wftools, mc
modules = [importlib, slater, pyscftools,recipes, wftools, mc]
for m in modules:
    importlib.reload(m)
from pyscf.dft import numint, libxc

mol, mf = pyscftools.recover_pyscf(scf_checkfile)
wfs, to_opt = wftools.generate_wf(
        mol, mf, mc=None, jastrow=None, jastrow_kws=None, slater_kws=None
        )


nconf = 100
configs = mc.initial_guess(mol, nconf)
# signs, phis = wfs.recompute(configs)



In [23]:
# value
# import pyqmc.gpu as gpu
# nelec = 2
# e = 0
# aos = wfs.orbitals.aos("GTOval_sph", configs)
# wfs._aovals = aos.reshape(-1, nconf, nelec, aos.shape[-1])
# s = 0
# begin = wfs._nelec[0] * s
# end = wfs._nelec[0] + wfs._nelec[1] * s
# mo = wfs.orbitals.mos(wfs._aovals[:, :, begin:end, :], s)
# mo_vals = gpu.cp.swapaxes(mo[:, :, wfs._det_occup[s]], 1, 2)

In [27]:
# gradient
# e = 0
# epos = configs.electron(e)
# s = int(e >= wfs._nelec[0])
# aograd = wfs.orbitals.aos("GTOval_sph_deriv1", epos)
# mograd = wfs.orbitals.mos(aograd, s)
# mograd_vals = mograd[:, :, wfs._det_occup[s]]

In [28]:
mograd_vals.shape

(4, 100, 1, 1)

In [36]:
# val(x1), grad(x3), laplacian(x1)
# e = 0
# epos = configs.electron(e)
s = int(e >= wfs._nelec[0])
ao = wfs.orbitals.aos("GTOval_sph_deriv2", epos)
ao_val = ao[:, 0, :, :]
ao_lap = gpu.cp.sum(ao[:, [4, 7, 9], :, :], axis=1)

ao = gpu.cp.concatenate(
            [ao[:, 0:4, ...], ao[:, [4, 7, 9], ...].sum(axis=1, keepdims=True)], axis=1
        )
mo = wfs.orbitals.mos(ao, s)
mo_vals = mo[:, :, wfs._det_occup[s]]

In [39]:
kinetic = -mo_vals[-1]/(2*mo_vals[-1])
kinetic.shape

(100, 1, 1)

In [44]:
mo_energy = mf.mo_energy[s, wfs._det_occup[s]]
mo_energy.shape
# dir(mf)

(1, 1)

In [56]:
dm = mf.make_rdm1()
ao_value = numint.eval_ao(mol, configs.configs[:,e,:])
rho_u = numint.eval_rho(mol, ao_value, dm[0], xctype='LDA')
rho_d = numint.eval_rho(mol, ao_value, dm[1], xctype='LDA')
xc = 'LDA,VWN'
excd, vxcs  = libxc.eval_xc(xc, np.array([rho_u, rho_d]), spin=1)[:2]
vxc = np.zeros(nconf)
vj = np.zeros(nconf)
vxc += vxcs[0][:,s]
vj += np.einsum('pij,sij->p', mol.intor('int1e_grids', grids=configs.configs[:,e,:]), dm)
vee = vxc + vj
vee.shape

(100,)

In [59]:
kinetic[:,0,0]-vee

array([-1.89282077, -1.84934078, -1.47431872, -1.75034511, -1.64697852,
       -1.46433377, -1.31117807, -1.32216718, -2.1500698 , -1.84259816,
       -1.19015433, -2.06619407, -1.41689617, -1.68331818, -1.3702327 ,
       -1.65765474, -1.6425335 , -1.66651462, -1.3981506 , -1.1710012 ,
       -1.47700416, -1.45193394, -1.13144348, -1.85324143, -1.25169461,
       -1.11655931, -1.05552077, -1.93276162, -1.88777211, -1.97115482,
       -1.82357488, -1.26110164, -1.70621148, -1.26968534, -1.3993994 ,
       -1.45442079, -1.45080179, -2.04250899, -1.41259072, -1.62410177,
       -1.52755566, -1.8600861 , -1.54874208, -1.81866203, -1.21104103,
       -1.86672775, -1.35238269, -1.16392984, -1.64991859, -1.26056854,
       -1.39402639, -1.25993863, -1.14639941, -1.43904013, -2.19389576,
       -1.73804436, -1.08539934, -1.25431358, -1.93682018, -1.19495227,
       -1.70787412, -1.89121716, -2.36777612, -1.55025304, -2.20829124,
       -2.01420252, -1.67102783, -1.92854544, -1.51133399, -1.23

In [47]:
dir(mf)

['CASCI',
 'CASSCF',
 'CCSD',
 'CISD',
 'COSX',
 'DDCOSMO',
 'DFMP2',
 'DIIS',
 'Gradients',
 'Hessian',
 'MP2',
 'QCISD',
 'QMMM',
 'TDA',
 'TDHF',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_chkfile',
 '_eigh',
 '_eri',
 '_finalize',
 '_is_mem_enough',
 '_keys',
 '_nelec',
 '_opt',
 '_transfer_attrs_',
 'analyze',
 'apply',
 'as_scanner',
 'build',
 'callback',
 'canonicalize',
 'check_convergence',
 'check_sanity',
 'chkfile',
 'conv_check',
 'conv_tol',
 'conv_tol_grad',
 'converged',
 'convert_from_',
 'copy',
 'damp',
 'ddCOSMO',
 'density_fit',
 'det_ovlp',
 'diis',
 'diis_file',
 'diis_space',
 'diis_space_rollback',
 'diis_s